In [204]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
import seaborn
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from pycaret.classification import *
from pycaret.classification import evaluate_model
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import selenium
from selenium import webdriver
import random
from selenium.webdriver.common.by import By
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
pd.set_option('display.max_columns', 100)

In [277]:
def kelly(p, b, weight):
    stake = round((p * (b - 1) - (1 - p)) / (b - 1), 4) * weight
    return stake if stake > 0.0 else 'no value'

In [301]:
today = f"{datetime.now():%Y%m%d}"
setup_df = pd.read_csv(f'data_{today}.csv', index_col = [0])
setup_df = setup_df[setup_df['odds_checker_sum'] > 1.02]
print(setup_df.shape)

(275, 51)


In [302]:
setup_df = setup_df[['Expected goals (xG)', 'Total shots', 'Big chances','Accurate passes percentage',
        'xG on target (xGOT)', 'Passes', 'Own half', 'Opposition half','Red cards','score', 'posession',
        '0', '1', '2','target']]

In [287]:
setup_df

,expected_goals_(xg)_diff,expected_goals_(xg)_sum,total_shots_diff,total_shots_sum,big_chances_diff,big_chances_sum,big_chances_missed_diff,big_chances_missed_sum,accurate_passes_diff,accurate_passes_sum,...,xg_penalty_diff,xg_penalty_sum,odds_predict,std_0,std_1,std_2,med_0,med_1,med_2,target
0,0.89,1.11,7.0,9.0,1.0,1.0,0.0,0.0,77.0,309.0,...,NaN,NaN,0,0.075452,0.045693,0.212633,2.30,2.387500,4.20000,0
1,-1.21,1.47,-5.0,15.0,-1.0,1.0,0.0,0.0,-54.0,262.0,...,NaN,NaN,2,0.286912,1.108553,0.021276,4.75,13.000000,1.25000,2
2,-0.74,1.08,-3.0,9.0,-1.0,1.0,0.0,0.0,-140.0,422.0,...,NaN,NaN,2,3.103652,20.120048,0.005754,20.00,56.000000,1.02439,2
3,0.37,0.49,4.0,8.0,1.0,1.0,1.0,1.0,56.0,380.0,...,NaN,NaN,2,0.093617,0.235317,0.018924,3.50,6.500000,1.60000,1
4,-2.94,3.22,-12.0,18.0,-3.0,3.0,0.0,0.0,-242.0,366.0,...,NaN,NaN,2,262.082237,309.896578,0.033394,226.00,201.000000,1.04000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,-0.31,0.69,1.0,9.0,-2.0,2.0,-1.0,1.0,-45.0,385.0,...,NaN,NaN,2,0.114002,0.271888,0.022401,4.00,9.000000,1.40000,1
282,0.39,1.23,5.0,13.0,1.0,5.0,0.0,4.0,93.0,327.0,...,NaN,NaN,1,0.746496,0.013245,5.390733,9.00,1.083333,34.00000,1
283,0.39,1.09,1.0,11.0,1.0,1.0,1.0,1.0,-55.0,289.0,...,NaN,NaN,1,0.115587,0.038570,0.329347,3.10,1.727273,5.22500,2
284,0.08,0.50,0.0,8.0,1.0,1.0,1.0,1.0,-30.0,290.0,...,NaN,NaN,2,0.085853,0.116416,0.073947,2.90,3.376471,2.25000,2


In [303]:
grid = setup(data=setup_df, target='target', session_id = 123)
#             ignore_features = ['full_odds'])


,Description,Value
0,session_id,123
1,Target,target
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(275, 15)"
5,Missing Values,True
6,Numeric Features,14
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [304]:
best = compare_models()
tuned = tune_model(best)
print(tuned)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5000,0.0000,0.4074,0.3857,0.4352,0.1837,0.1952
1,0.7000,0.0000,0.6407,0.6917,0.6714,0.5294,0.5494
2,0.6316,0.0000,0.5648,0.6924,0.6101,0.3843,0.3988
3,0.6842,0.0000,0.6204,0.6699,0.6649,0.4771,0.4870
4,0.5789,0.0000,0.5833,0.5684,0.5719,0.3244,0.3259
5,0.6316,0.0000,0.5463,0.6211,0.6238,0.4089,0.4108
6,0.6316,0.0000,0.5463,0.6041,0.6113,0.3955,0.4014
7,0.9474,0.0000,0.9333,0.9561,0.9468,0.9174,0.9214
8,0.5789,0.0000,0.5000,0.4774,0.4957,0.3091,0.3638


RidgeClassifier(alpha=7.73, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=123, solver='auto',
                tol=0.001)


In [275]:
xgb = create_model('xgboost')
tuned = tune_model(xgb)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5789,0.7085,0.5361,0.5697,0.5567,0.3333,0.3447
1,0.5263,0.7019,0.4583,0.4825,0.4842,0.2434,0.2584
2,0.5789,0.7686,0.5278,0.4781,0.5228,0.3420,0.3532
3,0.5263,0.7453,0.4694,0.4820,0.4909,0.2500,0.2585
4,0.5789,0.8759,0.5417,0.4902,0.5175,0.3532,0.3746
5,0.7222,0.7949,0.6917,0.7207,0.7179,0.5652,0.5681
6,0.6111,0.6689,0.5889,0.6111,0.6111,0.3762,0.3762
7,0.8889,0.9211,0.8500,0.8944,0.8864,0.8173,0.8218
8,0.7222,0.7935,0.6528,0.6949,0.6965,0.5545,0.5634


In [308]:
evaluate_model(tuned)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [307]:
df

,med_0,med_1,med_2,target,Label,Score_0,Score_1,Score_2,draw,home,away
0,2.500,3.200000,2.700,0,0,0.5279,0.0332,0.4389,0.053300,0.000000,0.02720
1,4.200,8.000000,1.400,2,2,0.0048,0.0005,0.9947,0.000000,0.000000,0.24535
2,2.625,2.500000,3.300,1,0,0.5711,0.0957,0.3333,0.076800,0.000000,0.01085
3,3.100,1.650000,7.000,0,1,0.0329,0.9642,0.0029,0.000000,0.227275,0.00000
4,21.000,1.024390,67.000,1,1,0.0003,0.9969,0.0028,0.000000,0.217450,0.00000
...,...,...,...,...,...,...,...,...,...,...,...
81,4.300,1.363636,10.000,1,1,0.1783,0.8136,0.0081,0.000000,0.075250,0.00000
82,2.500,2.250000,4.000,1,0,0.7268,0.0978,0.1754,0.136175,0.000000,0.00000
83,3.200,4.000000,1.950,1,2,0.0256,0.0282,0.9462,0.000000,0.000000,0.22240
84,2.375,3.400000,2.625,0,0,0.9056,0.0145,0.0799,0.209225,0.000000,0.00000


In [309]:
predicted_df = predict_model(tuned, raw_score = True)
z = predicted_df
predicted_df = predicted_df[['0','1','2'] + list(predicted_df.columns[-5:])]
bo_po_dha = list(zip(predicted_df.columns[0:3],predicted_df.columns[-3:],['draw','home','away']))
for book_odds, pred_odds, dha in bo_po_dha:
    predicted_df[dha] = predicted_df.apply(lambda x: kelly(x[pred_odds], x[book_odds], 0.25), axis=1)
predicted_df.to_csv('pred_profits.csv')
df = predicted_df
dha = {'draw':'0','home':'1','away':'2'}
df = df.replace('no value', 0)
df[['draw','home','away']] = df[['draw','home','away']].astype(float)
for word, number in dha.items():
    df[f'{word}_profits'] = np.where(df['final_score'] == int(number),
                                    df[word] * (df[number] - 1) * 100,
                                    -100*df[word])
df['profits'] = df[df.columns[-3:]].sum(axis = 1)
p = round(np.sum(df["profits"]), 3)
df = df.round(3)
df.to_csv(f'profits_{datetime.now():%Y%m%d}.csv')
print(f'number of bets made: {df.shape[0]}')
print(f'profit on $100 bankroll: ${p}')
print(df.sum()[-4:-1])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Ridge Classifier,0.5663,0,0.5129,0.5558,0.5283,0.3169,0.3426


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [310]:
predicted_df

,0,1,2,0,1,2,target,Label
0,2.600000,2.375000,3.500000,2.600000,2.375000,3.500000,2,1
1,36.000000,1.006667,95.000000,36.000000,1.006667,95.000000,0,1
2,3.400000,7.000000,1.591667,3.400000,7.000000,1.591667,2,2
3,5.000000,14.000000,1.250000,5.000000,14.000000,1.250000,2,2
4,2.750000,1.833333,5.500000,2.750000,1.833333,5.500000,1,1
...,...,...,...,...,...,...,...,...
78,2.750000,2.100000,4.000000,2.750000,2.100000,4.000000,1,1
79,2.875000,1.875000,5.000000,2.875000,1.875000,5.000000,0,1
80,3.500000,1.531667,7.500000,3.500000,1.531667,7.500000,0,1
81,4.333333,1.363636,10.000000,4.333333,1.363636,10.000000,1,1


In [253]:
df.sort_values(by = 'Score_0')

,0,1,2,target,Label,Score_0,Score_1,Score_2,draw,home,away,draw_profits,home_profits,away_profits,profits
11,10.000,1.083,29.000,1,1,0.006,0.984,0.010,0.000,0.197,0.000,-0.000,1.645,-0.000,1.645
3,25.000,1.020,76.000,1,1,0.006,0.888,0.106,0.000,0.000,0.024,-0.000,0.000,-2.352,-2.352
48,4.750,1.250,13.000,1,1,0.009,0.943,0.048,0.000,0.179,0.000,-0.000,4.481,-0.000,4.481
2,3.400,1.667,5.800,1,1,0.011,0.954,0.035,0.000,0.221,0.000,-0.000,14.742,-0.000,14.742
78,15.000,1.030,59.000,1,1,0.011,0.973,0.016,0.000,0.022,0.000,-0.000,0.067,-0.000,0.067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,2.500,3.200,2.700,0,0,0.736,0.133,0.131,0.140,0.000,0.000,20.992,-0.000,-0.000,20.992
10,2.700,2.900,2.750,1,0,0.770,0.037,0.193,0.159,0.000,0.000,-15.875,0.000,-0.000,-15.875
54,2.250,3.500,2.800,2,0,0.793,0.060,0.148,0.157,0.000,0.000,-15.668,-0.000,0.000,-15.668
76,2.625,7.500,1.727,2,0,0.806,0.083,0.110,0.172,0.000,0.000,-17.185,-0.000,0.000,-17.185


In [196]:
z[['0','1','2','Score_0','Score_1','Score_2', 'final_score']]

,0,1,2,Score_0,Score_1,Score_2,final_score
0,2.350000,3.200000,3.000000,0.9951,0.0043,0.0007,0
1,3.673529,6.000000,1.533333,0.1092,0.1388,0.7520,2
2,4.800000,15.000000,1.250000,0.0004,0.0000,0.9996,0
3,2.600000,5.000000,2.050000,0.0080,0.9843,0.0077,0
4,2.300000,2.522727,3.823077,0.5503,0.0008,0.4489,0
...,...,...,...,...,...,...,...
66,3.275000,1.775000,4.500000,0.3791,0.5606,0.0603,2
67,3.500000,10.000000,1.444444,0.0119,0.0100,0.9782,2
68,21.000000,1.022500,67.000000,0.0000,0.9996,0.0004,1
69,2.250000,6.000000,2.100000,0.9690,0.0159,0.0150,2
